**Название проекта:**

Поиск токсичных комментариев для их дальнейшей модерации.
____

**Цель исследования:**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

***Задачи исследования:***

*    Обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении имеется набор данных с разметкой о токсичности правок.
*    Построить модель со значением метрики качества F1 не меньше 0.75.

**Исходные данные:**

*    В распоряжении имеется набор <a href="https://code.s3.yandex.net/datasets/toxic_comments.csv"> данных </a> с разметкой о токсичности правок.


***Описание данных:***

Таблица **toxic_comments.csv** (данные с разметкой):

**Признаки:**

*    text — текст комментария

**Целевой признак:**

*    toxic - токсичный комментарий или нет

____

**Данное исследование разделим на несколько частей.**

<a href='#link_1'> ***Часть 1. Изучение общей информации и подготовка данных:***</a>

*   Изучение файлов с данными, получение общей информации, загрузка библиотек.
*   Поиск явных дубликатов
*   Нахождение пропусков поиск их причин.
*   Оценка типов данных


<a href='#link_2'> ***Часть 2. Обучение моделей***</a>

*    <a href='#link_2.1'>2.1 Модель DecisionTreeClassifier.</a>
*    <a href='#link_2.2'>2.2 Модель LGBMClassifier.</a>
*    <a href='#link_2.3'>2.3 Модель LogisticRegression.</a>  

<a href='#link_3'> ***Часть 3. Выводы.***</a>


<a id='link_1'> </a>
## Изучение и подготовка данных

In [46]:
# Импортируем библиотеки
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import winsound

import optuna
import time

import nltk
import re

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet','stopwords','punkt','averaged_perceptron_tagger')


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import RidgeClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

STATE = np.random.RandomState(12345)

In [47]:
def sound():
    '''
    Функция для подачи звука.
    Нужно для сигнализации об окончании выполнения кода.
    '''
    duration = 1000 # миллисекунды
    freq = 440 # Гц
    winsound.Beep(freq, duration)

In [49]:
# Функция для получения первичной информации об исходных данных
def get_info(df):
    '''
    Функция для получения первичной информации об исходных данных:
       - типы данных,
       - количество пропусков,
       - кол-во дубликатов, 
       - характеристики числовых столбцов
    '''
    print(100 * "-")
    print('Общее описание полученных данных:')
    print(100 * "-")
    df.info()
    print(100 * "-")
    print('Описание количественных переменных:')
    print(100 * "-")    
    display(df.describe(datetime_is_numeric=True))
    print(100 * "-")
    print(f'Количество полных дубликатов в таблице = {df.duplicated().sum()}')

In [50]:
# Откроем данные из файла
df_raw = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [51]:
df = df_raw.copy()

In [52]:
# df = df.sample(500)

In [53]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [54]:
get_info(df)

----------------------------------------------------------------------------------------------------
Общее описание полученных данных:
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
----------------------------------------------------------------------------------------------------
Описание количественных переменных:
----------------------------------------------------------------------------------------------------


,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


----------------------------------------------------------------------------------------------------
Количество полных дубликатов в таблице = 0


In [56]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [57]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(row):
    '''
    Функция очищает текст от лишних символов и пробелов, кроме латиницы
    и лематизирует текст.
    '''
    text = row['text']

    # удалим ссылки на сайты типа www.yandex.ru или http://stackoverflow.com
    text = re.sub(r'(https?://[^\s]+)', ' ', text)

    # удалим адреса электронной почты типа this_is.my-email@yandex-super.ru
    text = re.sub(r'([\w.+-]+@[\w-]+\.[\w.-]+)', ' ', text)

    # удалим обращения к пользователям типа @Maxim
    text = re.sub(r'(@[\w-]+[\w.-]+)', ' ', text)

    # удалим все символы кроме символов английского алфавита (a-zA-Z)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
#     text = ' '.join(text.split())

    # лемматизируем текст
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    text = ' '.join(text)

    return text.lower()

In [58]:
# Лемматизируем текст в таблице с сохраними в отдельный столбец.
df['lemm_text'] = df.apply(lemmatize_text, axis=1)

In [59]:
# df.to_csv('datasets/toxic_comments_lemm.csv', index=False)

In [60]:
# # Откроем данные из файла используя функцию 'open_file'
# df = open_file('datasets/toxic_comments_lemm.csv')
# # df = df.sample(20000)

In [61]:
df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [62]:
# Разобъем на train и test
train, test = train_test_split(df, test_size=0.25, random_state=STATE)

In [63]:
# Загрузим стоп-слова
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [65]:
# Создадим признаки
features_train = count_tf_idf.fit_transform(corpus_train)
features_test = count_tf_idf.transform(corpus_test)

In [66]:
# Выделим целевой признак
target_train = train['toxic']
target_test = test['toxic']

In [67]:
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((119678, 130411), (119678,), (39893, 130411), (39893,))

<a id='link_2'> </a>
## Обучение моделей.

In [68]:
def print_result(result, params = 0):
    '''
    Функция выводит на печать результаты работы модели
       - result - словарь(имя модели, время обучения, Время предсказания, RMSE)
       - params - гиперпараметры модели
    '''
    print('-' * 100)
    print(f'Для модели {result["model"]}')
#     print(f'Гиперпараметры: {params}')
    print(f'f1 = {result["f1"]:.3f}')
    print(f'Время обучения: {result["learning_time"]:0.1f} секунд')
    print('-' * 100)

In [69]:
def optuna_hyper(X_train, y_train, model, params, cv=5, n_trials=10, scoring='f1'):  
    '''
    Функция для подбора гиперпараметров с помощью OptunaSearchCV
    
    Пример:
        params = {  'max_depth': optuna_int(5, 13),
                    'min_samples_leaf': optuna_int(5, 13),
                    'class_weight': optuna_cat(['balanced'])}

        optuna_hyper(X, y, DecisionTreeClassifier(), params, cv=3, n_trials=5, scoring='f1')
    '''
    # t0 - время начало отсчета
    t0 = time.perf_counter()
    
    optuna_search = optuna.integration.OptunaSearchCV(
        estimator=model, 
        param_distributions=params, 
        cv=cv, n_trials=n_trials,
        scoring=scoring, random_state=STATE)
    
    optuna_search.fit(X_train, y_train)
    
    # Печать лучших параметров и метрики
    print(f'best params: {optuna_search.best_params_}')
    print('score = {:.2f}'.format(optuna_search.best_score_))
    
    # Печать времени подбора параметров
    print(f'elapsed time: {time.perf_counter() - t0:0.1f}s')
    
    return optuna_search.best_params_

optuna_cat = optuna.distributions.CategoricalDistribution
optuna_int = optuna.distributions.IntUniformDistribution
optuna_float = optuna.distributions.LogUniformDistribution

In [70]:
def model_fit(features_train, features_test, target_train, target_test, model_name, params):
    
    """
    Функция обучает модель.
    На вход подается обучающий набор данных features_train, target_train.
    Также на вход подаются или валидационный набор данных features_valid, target_valid,
    или тестировочный набор данных features_test, target_test.
    
    На выходе функция выдает результаты обучения и предсказания.
    
    Пример: model_fit(X_train, X_test, y_train, y_test, DecisionTreeClassifier, best_params)
    
    """
    t0 = time.perf_counter()
    
    model = model_name(**params) 
    
    # Обучение модели
    model.fit(features_train, target_train)
    
    t1 = time.perf_counter()
    
    # Получение предсказаний
    prediction = model.predict(features_test)
    
    # Расчет метрики
    score = f1_score(target_test, prediction)
    
    result = {'model': model_name.__name__, 'learning_time': t1-t0, 'f1': score}
    
    print_result(result, params)
    
    return result, prediction

<a id='link_2.1'> </a>
### DecisionTreeClassifier

In [71]:
# Подберем гиперпараметры
params = {'max_depth': optuna_int(5, 13),
          'min_samples_leaf': optuna_int(5, 13),
          'class_weight': optuna_cat(['balanced'])
         }

best_params_dtc = optuna_hyper(features_train, target_train, 
                               DecisionTreeClassifier(), params, 
                               cv=3, n_trials=10, scoring='f1')

<ipython-input-69-fd6eb69cea57>:15: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2022-05-21 15:06:32,301] A new study created in memory with name: no-name-59da05b8-e0d2-4de3-ae69-c10a394f838c
[I 2022-05-21 15:07:21,048] Trial 0 finished with value: 0.5833653074528389 and parameters: {'max_depth': 12, 'min_samples_leaf': 6, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5833653074528389.
[I 2022-05-21 15:08:03,849] Trial 1 finished with value: 0.5576234263213142 and parameters: {'max_depth': 9, 'min_samples_leaf': 12, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5833653074528389.
[I 2022-05-21 15:08:45,551] Trial 2 finished with value: 0.5441242791770976 and parameters: {'max_depth': 8, 'min_samples_leaf': 13, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5833653074528389.
[I 2022-05-21 15:09:30,059] Trial 3 finished with

best params: {'max_depth': 13, 'min_samples_leaf': 8, 'class_weight': 'balanced'}
score = 0.59
elapsed time: 460.2s


In [72]:
# Обучим модель и проверим на тесте
result_dtc, prediction_dtc = model_fit(features_train, features_test,
                               target_train, target_test,
                               DecisionTreeClassifier,
                               best_params_dtc)

----------------------------------------------------------------------------------------------------
Для модели DecisionTreeClassifier
f1 = 0.592
Время обучения: 9.5 секунд
----------------------------------------------------------------------------------------------------


<a id='link_2.2'> </a>
### LGBMClassifier

In [73]:
# Подберем гиперпараметры
params = {'n_estimators': optuna_int(50, 200),
          'learning_rate': optuna_float(0.01, 0.6),
          'max_depth': optuna_int(5, 15),
          'class_weight': optuna_cat(['balanced'])
         }

best_params_lgbmc = optuna_hyper(features_train, target_train, 
                                 LGBMClassifier(), params, 
                                 cv=3, n_trials=10, scoring='f1')

<ipython-input-69-fd6eb69cea57>:15: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2022-05-21 15:14:21,732] A new study created in memory with name: no-name-9f6c1112-4f27-4bb8-a2c7-2dfe1aef25db
[I 2022-05-21 15:15:18,625] Trial 0 finished with value: 0.6847420472085343 and parameters: {'n_estimators': 76, 'learning_rate': 0.038209344529222006, 'max_depth': 11, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.6847420472085343.
[I 2022-05-21 15:16:16,441] Trial 1 finished with value: 0.7390867391884752 and parameters: {'n_estimators': 197, 'learning_rate': 0.1992497015758169, 'max_depth': 5, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.7390867391884752.
[I 2022-05-21 15:17:15,772] Trial 2 finished with value: 0.687319181831147 and parameters: {'n_estimators': 84, 'learning_rate': 0.03963890873211134, 'max_depth': 11, 'class_weight': 'balan

best params: {'n_estimators': 189, 'learning_rate': 0.29536558957272674, 'max_depth': 9, 'class_weight': 'balanced'}
score = 0.75
elapsed time: 664.8s


In [74]:
# Обучим модель и проверим на тесте
result_lgbmc, prediction_lgbmc = model_fit(features_train, features_test, 
                                           target_train, target_test, 
                                           LGBMClassifier, 
                                           best_params_lgbmc)

----------------------------------------------------------------------------------------------------
Для модели LGBMClassifier
f1 = 0.748
Время обучения: 35.5 секунд
----------------------------------------------------------------------------------------------------


<a id='link_2.3'> </a>
### LogisticRegression

In [75]:
# Подберем гиперпараметры
params = {'C': optuna_float(0.1, 20),
          'class_weight': optuna_cat(['balanced']),
          'solver': optuna_cat(['liblinear'])
         }

best_params_lr = optuna_hyper(features_train, target_train, 
                              LogisticRegression(), params,
                              cv=3, n_trials=10, scoring='f1')

<ipython-input-69-fd6eb69cea57>:15: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2022-05-21 15:26:02,702] A new study created in memory with name: no-name-99e7c122-9297-478d-9a11-d4c512c8847c
[I 2022-05-21 15:26:07,185] Trial 0 finished with value: 0.7443760372135535 and parameters: {'C': 1.1041373344179033, 'class_weight': 'balanced', 'solver': 'liblinear'}. Best is trial 0 with value: 0.7443760372135535.
[I 2022-05-21 15:26:10,862] Trial 1 finished with value: 0.7381880088541236 and parameters: {'C': 0.7638728341455373, 'class_weight': 'balanced', 'solver': 'liblinear'}. Best is trial 0 with value: 0.7443760372135535.
[I 2022-05-21 15:26:14,825] Trial 2 finished with value: 0.7395792882423585 and parameters: {'C': 0.8268607972695081, 'class_weight': 'balanced', 'solver': 'liblinear'}. Best is trial 0 with value: 0.7443760372135535.
[I 2022-05-21 15:26:17,82

best params: {'C': 5.85214914921416, 'class_weight': 'balanced', 'solver': 'liblinear'}
score = 0.76
elapsed time: 57.3s


In [76]:
# Обучим модель и проверим на тесте
result_lr, prediction_lr = model_fit(features_train, features_test,
                                     target_train, target_test,
                                     LogisticRegression,
                                     best_params_lr)

----------------------------------------------------------------------------------------------------
Для модели LogisticRegression
f1 = 0.762
Время обучения: 3.7 секунд
----------------------------------------------------------------------------------------------------


<a id='link_3'> </a>
## Выводы.

Выведем результаты отработки моделей

In [77]:
# для модели DecisionTreeClassifier
print_result(result_dtc, best_params_dtc)

# для модели LGBMClassifier
print_result(result_lgbmc, best_params_lgbmc)

# для модели LogisticRegression
print_result(result_lr, best_params_lr)

----------------------------------------------------------------------------------------------------
Для модели DecisionTreeClassifier
f1 = 0.592
Время обучения: 9.5 секунд
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Для модели LGBMClassifier
f1 = 0.748
Время обучения: 35.5 секунд
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Для модели LogisticRegression
f1 = 0.762
Время обучения: 3.7 секунд
----------------------------------------------------------------------------------------------------


In [78]:
# Создадим словарь с готовыми ключами и пустыми списками в нем
results = {i: [] for i in result_dtc.keys()}
results

{'model': [], 'learning_time': [], 'f1': []}

In [79]:
# Соберем все результаты в словарь 'results'
for result in [result_dtc, result_lgbmc, result_lr]:
    for key, value in result.items():
        results[key].append(value)
results

{'model': ['DecisionTreeClassifier', 'LGBMClassifier', 'LogisticRegression'],
 'learning_time': [9.488682000001063, 35.49198260000048, 3.659893699999884],
 'f1': [0.591978287092883, 0.7479249573621375, 0.76207473508087]}

In [80]:
# Соберем результаты в таблицу
results = pd.DataFrame(results)
results

,model,learning_time,f1
0,DecisionTreeClassifier,9.488682,0.591978
1,LGBMClassifier,35.491983,0.747925
2,LogisticRegression,3.659894,0.762075


**Вывод:**

В итоге мы видим, что деревянные модели показывают себя не с лучшей стороны в решении данной задачи. При этом случайный лес показывает еще хуже результат. 

Близкий к заданной метрике показывает результат модель LGBMClassifier.  При этом имеет самое высокое время обучение. Можно попробовать добиться нужной метрики, подкрутив немного еще гиперпараметры, но при этом увеличится время подбора гиперпараметров.

Самые лучшие показатели дает LogisticRegression как по времени обучения, так и по метрике f1 = 0.76. 